# Install necessary dependencies

In [ ]:
!pip install --upgrade efficientnet-pytorch
!pip install --upgrade pretrainedmodels
!pip install git+https://github.com/qubvel/segmentation_models.pytorch
!pip install git+https://github.com/michal-nahlik/FastFCN

# Prepare

## Imports

In [ ]:
import gc
import os
import cv2
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as albu

from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer, required
from torch.utils.data import Dataset, DataLoader

import pretrainedmodels
from efficientnet_pytorch import EfficientNet
from segmentation_models_pytorch import Unet, FPN, PSPNet
from encoding.models.encnet import EncNet
from encoding.models.deeplabv3 import DeepLabV3

## Set seeds

In [ ]:
seed = 2019
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

## Dataset

In [ ]:
class CloudDataset(Dataset):
    def __init__(self, list_IDs=None, rles_df = None, data_folder = None, transforms=None,
                dim=(1400, 2100), reshape=(320, 480)):
        self.list_IDs = list_IDs
        self.rles_df = rles_df
        self.data_folder = data_folder
        self.transforms = transforms
        self.dim = dim
        self.reshape = reshape
        

    def __getitem__(self, idx):
        ID = self.list_IDs.iloc[idx]
        img_path = self.data_folder + ID
        img = self.load_rgb(img_path)
        mask = None
        
        if self.reshape is not None:
            img = np_resize(img, self.reshape)
            
        if self.rles_df is not None:
            image_df = self.rles_df[self.rles_df['ImageId'] == ID]
            rles = image_df['EncodedPixels'].values
            
            if self.reshape is not None:
                mask = build_masks(rles, input_shape=self.dim, reshape=self.reshape)
            else:
                mask = build_masks(rles, input_shape=self.dim)
        
        if self.transforms is not None:
            augmented = self.transforms(image=img, mask=mask)
            img  = augmented["image"]
            mask = augmented["mask"]
        
        if mask is None:
            return ID, img, []
        else:
            return ID, img, mask

    def __len__(self):
        return len(self.list_IDs)
    
    def load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

## Helper functions

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dice(img1, img2):
    img1 = img1 > 0.5
    img2 = img2 > 0.5
    img1 = np.asarray(img1).astype(np.bool)
    img2 = np.asarray(img2).astype(np.bool)

    intersection = np.logical_and(img1, img2)

    return 2.0 * intersection.sum() / (img1.sum() + img2.sum())

## rle, mask, image manipulation

In [ ]:
def np_resize(img, input_shape):
    """
    Reshape a numpy array, which is input_shape=(height, width), 
    as opposed to input_shape=(width, height) for cv2
    """
    height, width = input_shape
    return cv2.resize(img, (width, height))

def mask2rle(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    """
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape, reshape=None):
    depth = len(rles)
    if reshape is None:
        masks = np.zeros((*input_shape, depth))
    else:
        masks = np.zeros((*reshape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            if reshape is None:
                masks[:, :, i] = rle2mask(rle, input_shape)
            else:
                mask = rle2mask(rle, input_shape)
                reshaped_mask = np_resize(mask, reshape)
                masks[:, :, i] = reshaped_mask
    
    return masks

def build_rles(masks, thrs=None, reshape=None):
    width, height, depth = masks.shape
    
    rles = []
    
    for i in range(depth):
        mask = masks[:, :, i]
        if thrs is not None:
            mask = mask > thrs[i]
        
        if reshape:
            mask = mask.astype(np.float32)
            mask = np_resize(mask, reshape).astype(np.int64)
        
        rle = mask2rle(mask)
        rles.append(rle)
        
    return rles

def rle_area(rle):
    try:
        array = np.asarray([int(x) for x in rle.split()])
        lengths = array[1::2]
        return np.sum(lengths)
    except:
        return 0

## Post processing
Remove small masks and draw convex hull mask

In [ ]:
# https://www.kaggle.com/ratthachat/cloud-convexhull-polygon-postprocessing-no-gpu
def draw_convex_hull(mask, mode='convex'):
    
    img = np.zeros(mask.shape)
    contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if mode=='rect': # simple rectangle
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), -1)
        if mode=='convex': # minimum convex hull
            hull = cv2.convexHull(c)
            cv2.drawContours(img, [hull], 0, (255, 255, 255),-1)
        else: # minimum area rectangle
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (255, 255, 255),-1)
    return img/255.


def post_process(probability, threshold, min_size):
    """
    This is slightly different from other kernels as we draw convex hull here itself.
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    # don't remember where I saw it
    mask = (cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1])
    mask = draw_convex_hull(mask.astype(np.uint8))
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = component == c
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

## Transformations

In [ ]:
# Taken from newer version of albumentations, changed apply_to_mask transpose so the multilayer mask works
from albumentations.core.transforms_interface import BasicTransform

class ToTensorV2(BasicTransform):
    """Convert image and mask to `torch.Tensor`."""

    def __init__(self, always_apply=True, p=1.0):
        super(ToTensorV2, self).__init__(always_apply=always_apply, p=p)

    @property
    def targets(self):
        return {"image": self.apply, "mask": self.apply_to_mask}

    def apply(self, img, **params):
        return torch.from_numpy(img.transpose(2, 0, 1))

    def apply_to_mask(self, mask, **params):
        return torch.from_numpy(mask.transpose(2, 0, 1))

    def get_transform_init_args_names(self):
        return []

    def get_params_dependent_on_targets(self, params):
        return {}

In [ ]:
import cv2

from albumentations import (
    Compose, VerticalFlip, HorizontalFlip, ShiftScaleRotate, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ToFloat, ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,OpticalDistortion,RandomSizedCrop
)

AUGMENTATIONS_TEST = Compose([
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)

AUGMENTATIONS_TEST_TTA_1 = Compose([
    HorizontalFlip(p=1),
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)

AUGMENTATIONS_TEST_TTA_2 = Compose([
    VerticalFlip(p=1),
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)

AUGMENTATIONS_TEST_TTA_3 = Compose([
    HorizontalFlip(p=1),
    VerticalFlip(p=1),
    ToFloat(max_value=1),
    ToTensorV2()
],p=1)

# Load and prepare data

In [ ]:
class_names = ["Fish", "Flower", "Gravel", "Sugar"]

In [ ]:
path = '../input/understanding_cloud_organization/'
path_train = path + 'train_images/'
path_test = path + 'test_images/'

train_on_gpu = torch.cuda.is_available()

In [ ]:
train_df = pd.read_csv(path + 'train.csv')
train_df['ImageId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['MaskArea'] = train_df['EncodedPixels'].apply(lambda x: rle_area(x))
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg({'hasMask' : np.sum, 'MaskArea': list}).reset_index()
print(np.shape(mask_count_df))
mask_count_df.head()

## Split based on mask area

In [ ]:
mask_areas = np.stack(mask_count_df['MaskArea'].values)
mask_areas = mask_areas > 0
train_idx, val_idx = train_test_split(mask_count_df['ImageId'], stratify=mask_areas, random_state=seed, test_size=0.2)

In [ ]:
train_mask_areas = np.stack(mask_count_df.iloc[train_idx.index]['MaskArea'].values)
val_mask_areas   = np.stack(mask_count_df.iloc[val_idx.index]['MaskArea'].values)

f, ax = plt.subplots(nrows=2, ncols=4, figsize=(20,8))
for i in range(0,4):
    sns.distplot(train_mask_areas[:,i], ax=ax[0,i]).set_title(class_names[i])
    sns.distplot(val_mask_areas[:,i], ax=ax[1,i])

In [ ]:
print('Training set size: {}'.format(len(train_idx)))
print('Validation set size: {}'.format(len(val_idx)))

## Create data loader

In [ ]:
# define dataset and dataloader
num_workers = 4
bs = 8

valid_dataset = CloudDataset(list_IDs = val_idx,   rles_df=train_df, data_folder = path_train, transforms=AUGMENTATIONS_TEST)
valid_loader = DataLoader(valid_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

# Find best post processing params

## Helper functions for prediction

In [ ]:
def get_predictions(model, loader, limit, get_masks=True):
    valid_masks = []
    probabilities = []

    with torch.no_grad():
        for IDs, data, target in tqdm(loader):
            if train_on_gpu:
                data = data.to(device, dtype=torch.float)

            output = model(data)
            # EncNet has 3 outputs
            if (len(output) == 3):
                output = output[0].cpu().detach().numpy().astype(np.float16)
            # DeepLabV3 has 2 outputs
            elif (len(output) == 2):
                output = output[0].cpu().detach().numpy().astype(np.float16)
            else:
                output = output.cpu().detach().numpy().astype(np.float16)

            probabilities.extend(output)
            
            if get_masks:
                target = target.numpy().astype(np.uint8)
                valid_masks.extend(target)

            if np.shape(probabilities)[0] >= limit:
                break
                
    return valid_masks, probabilities


def predict_with_ttas(model, loader, limit=200):
    loader.dataset.transforms=AUGMENTATIONS_TEST
    valid_masks, probabilities = get_predictions(model, loader, limit)
    
    #valid_loader.dataset.transforms=AUGMENTATIONS_TEST_TTA_1
    #_, tmp = get_predictions(model, valid_loader, limit, False)
    #probabilities = np.sum([probabilities, tmp], axis=0)
    
    #valid_loader.dataset.transforms=AUGMENTATIONS_TEST_TTA_2
    #_, tmp = get_predictions(model, valid_loader, False)
    #probabilities = np.sum([probabilities, tmp], axis=0)

    #valid_loader.dataset.transforms=AUGMENTATIONS_TEST_TTA_3
    #_, tmp = get_predictions(model, valid_loader, False)
    #probabilities = np.sum([probabilities, tmp], axis=0)
    
    return valid_masks, probabilities


def load_model(path_model, device):
    model = torch.load(path_model)
    if train_on_gpu:
        model.to(device)

    model.eval()
    return model


def load_model_and_predict(path_model, device):
    model = load_model(path_model, device)
    return predict_with_ttas(model, valid_loader)

## Predict on validation set
Creates models and predicts on 200 images from validation dataset.

In [ ]:
device = torch.device("cuda:0")

In [ ]:
# FPN with EfficientB4 encoder
valid_masks, probabilities = load_model_and_predict('../input/clouds-pytorch-fpn/fpn_clouds_dice.pth', device)
prob_sum = probabilities

In [ ]:
# U-Net with EfficientB4 encoder
valid_masks, probabilities = load_model_and_predict('../input/clouds-pytorch-unet/unet_clouds_dice.pth', device)
prob_sum = np.sum([probabilities, prob_sum], axis=0)

In [ ]:
# EncNet with resnet50 encoder
valid_masks, probabilities = load_model_and_predict('../input/clouds-pytorch-encnet/encnet_clouds_dice.pth', device)
prob_sum = np.sum([probabilities, prob_sum], axis=0)

In [ ]:
# DeepLabV3 with resnet50 encoder
valid_masks, probabilities = load_model_and_predict('../input/clouds-pytorch-deeplabv3/deeplabv3_clouds_dice.pth', device)
prob_sum = np.sum([probabilities, prob_sum], axis=0)

In [ ]:
probabilities = prob_sum / 4

## Find best params

In [ ]:
def get_best_params(valid_masks, probabilities):
    class_params = {}

    for class_id in range(4):
        print(class_id)
        attempts = []
        for t in range(0, 100, 10):
            t /= 100
            for ms in range(1000,15000, 1000):
                masks = []
                for i in range(len(probabilities)):
                    probability = np.float32(probabilities[i][class_id][:][:])
                    probability = np_resize(probability, (350, 525))
                    predict, num_predict = post_process(sigmoid(probability), t, ms)
                    masks.append(predict)

                d = []
                for i in range(len(masks)):
                    target = valid_masks[i][class_id][:][:]
                    target = np_resize(target, (350, 525))
                    mask = masks[i][:][:]

                    if (target.sum() == 0) & (mask.sum() == 0):
                        d.append(1)
                    else:
                        d.append(dice(target, mask))

                attempts.append((t, ms, np.mean(d)))

        attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])
        attempts_df = attempts_df.sort_values('dice', ascending=False)
        print(attempts_df.head())
        best_threshold = attempts_df['threshold'].values[0]
        best_size = attempts_df['size'].values[0]

        class_params[class_id] = (best_threshold, best_size)
        
        sns.lineplot(x='threshold', y='dice', hue='size', data=attempts_df)
        plt.title('Threshold and min size vs dice for one of the classes')
        plt.show()
        
    return class_params

In [ ]:
class_params = get_best_params(valid_masks, probabilities)
class_params

In [ ]:
for i in range(0,2):    
    f, ax = plt.subplots(ncols=4, nrows=4, figsize=(20,8))

    ax[0][0].set_ylabel('Output')
    ax[1][0].set_ylabel('Threshold only')
    ax[2][0].set_ylabel('Post process')
    ax[3][0].set_ylabel('Target')
    
    for j in range(0, 4):
        p = np.float32(probabilities[i][j][:][:])
        p = np_resize(p, (350, 525))
        pp, num = post_process(sigmoid(p), class_params[j][0], class_params[j][1])
        
        ax[0][j].set_title(class_names[j])
        ax[0][j].imshow(p)
        ax[1][j].imshow(p > class_params[j][0])
        ax[2][j].imshow(pp)
        
        ax[3][j].imshow(np.float32(valid_masks[i][j][:][:]))

In [ ]:
# Clean up
del valid_loader, valid_dataset, probabilities, valid_masks, prob_sum
gc.collect()

# Predict on test data

## Prepare test dataset

In [ ]:
test_idx = os.listdir(path_test)
test_idx = pd.DataFrame(test_idx, columns={'ImageId'})

In [ ]:
num_workers = 4
bs = 8

test_dataset = CloudDataset(list_IDs = test_idx['ImageId'], rles_df=None, data_folder=path_test, transforms=AUGMENTATIONS_TEST)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, num_workers=num_workers)

## Load models

In [ ]:
# FPN with EfficientB4 encoder
model1 = load_model('../input/clouds-pytorch-fpn/fpn_clouds_dice.pth', device)
# U-Net with EfficientB4 encoder
model2 = load_model('../input/clouds-pytorch-unet/unet_clouds_dice.pth', device)
# EncNet with resnet50 encoder
model3 = load_model('../input/clouds-pytorch-encnet/encnet_clouds_dice.pth', device)
# DeepLabV3 with resnet50 encoder
model4 = load_model('../input/clouds-pytorch-deeplabv3/deeplabv3_clouds_dice.pth', device)

## Predict, post process and create rles

In [ ]:
image_labels = []
rles = []

with torch.no_grad():
    for IDs, data, _ in tqdm(test_loader):
        if train_on_gpu:
            data = data.to(device, dtype=torch.float)

        output1 = model1(data)
        output1 = output1.cpu().detach().numpy()
        
        output2 = model2(data)
        output2 = output2.cpu().detach().numpy()
        
        output3 = model3(data)[0]
        output3 = output3.cpu().detach().numpy()
        
        output4 = model4(data)[0]
        output4 = output4.cpu().detach().numpy()
        
        output = output1 + output2 + output3 + output4
        output = output / 4
        
        for i in range(0, len(IDs)):
            for j in range(0, 4):
                p = np.float32(output[i][j][:][:])
                p = np_resize(p, (350, 525))
                pp, num = post_process(sigmoid(p), class_params[j][0], class_params[j][1])
                rles.append(mask2rle(pp))
                image_labels.append(IDs[i] + '_' + class_names[j])

## Create and write submission file

In [ ]:
sub = pd.DataFrame({'Image_Label': image_labels, 'EncodedPixels': rles})
sub

In [ ]:
test_df = pd.read_csv('../input/understanding_cloud_organization/sample_submission.csv')
print('Sample submission lenght: {} \nTest submission length: {}'.format(len(test_df), len(sub))))

In [ ]:
sub.to_csv('clouds_submission.csv', index=False)